# Exploring NetCDF (*.nc) files
In this notebook we examine how the `xarray` package works with multidimensional NetCDF (`*.nc`) files. We discuss the structure and components of a NetCDF file (dimensions, variables, indexes, and attributes), as well as how to extract and summarize data across dimensions and variables held in the dataset.  

The dataset we'll use comes from NOAA's [Hindcast and Reanalysis Archives - Phase 2](https://polar.ncep.noaa.gov/waves/hindcasts/nopp-phase2.php).  

Specifically, we are using 10m partition data from January 1979 for the Mediterranean Sea ([link](https://polar.ncep.noaa.gov/waves/hindcasts/nopp-phase2/197901/partitions)). This [dataset](https://polar.ncep.noaa.gov/waves/hindcasts/nopp-phase2/197901/partitions/multi_reanal.partition.med_10m.197901.nc) will be available on the mapped class drive at `U:\859_data\multi_reanal.partition.med_10m.197901.nc`

In [ ]:
# Import packages
from pathlib import Path
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

## Read the NetCDF file into an `xarray` **dataset** object.

In [ ]:
# Set path to nc filed and import as an xarray dataset, then show the dataset object
nc_file = "V:\multi_reanal.partition.med_10m.197901.nc"
ds = xr.open_dataset(nc_file)
ds

## The components of an `xarray` dataset object:
### ➡️Dimensions
Dimensions define the axes or coordinate lengths that structure the dataset. In GIS, we are used to 2-dimensional datasets, e.g., the typical raster dataset, where values are associated with x and y geographic coordinates. But datasets can have additional dimensions. For example, if we have a separate raster for different time and depths, we'd have a 4 dimensional dataset. 

Our data has 4 dimensions:

In [ ]:
#Reveal the dimensions of the dataset
ds.dims

* There are **745 *time* steps** (`date: 745`)
* Each time step includes data for **301** points of ***latitude*** and **109** points of ***longitude*** (`longitude: 301, latitude: 109`)
* Each location at each time step is associated with **11 *partitions*** (`partition: 11`)

>  While the first three dimensions are intuitive, the fourth dimension, "partition" is specific to this dataset. Partitions refer to different spectral components of oceanic waves ([source](https://polar.ncep.noaa.gov/waves/workshop/pdfs/WW3-workshop-exercises-day4-wavetracking.pdf?utm_source=chatgpt.com)). For our purposes, we don't need to get into much detail beyond that; instead, perhaps just imagine that this dataset was collected by a set of 11 different sensor types, and the "partition" dimension represents data collected by each of these different sensors. 

In short, dimensions describe how the dataset is organized — e.g., time, space, or ensemble members. You could perhaps think of dimensions as the *axes of a multidimensional spreadsheet*.

### ➡️Data Variables

Data variables are the core measured or modeled quantities — the actual “data fields” stored along the dataset’s dimensions. 

In [ ]:
#Reveal the data variables of the dataset
ds.data_vars

Each data variable changes along on one or more dimensions.
For instance:
```python
depth(latitude, longitude)
significant_wave_height(date, partition, latitude, longitude)
```
* **depth** varies latitude and longitude, but it does not vary over time or by partition.
* **significant_wave_height** varies over date, location, and partition, so it’s a 4D variable: wave height for each partition of the spectrum at each time and location.

> Each variable is like a column in a data table, but in multiple dimensions.

### ➡️Indexes
While *dimensions* describe the axes along which our data are organized, *indexes* specify the measures along those axes — like labels for time, space, or category.

In [ ]:
#Reveal the indices of the dataset
ds.indexes

The above output shows the index values associated with each dimensions. 

For example, 
* The `date` dimension has an index for each hour from midnight on Jan 1, 1979 to midnight on Feb 1, 1979.
* The `longitude` dimension has 301 values that span from 7°W to 43°E.
* The `latitude` dimension has 109 values that span from 30°N to 48°N.
* The `partition` dimension has no meaningful index values.

We can use these index values to extract data in meaningful ways, e.g. by time and location. 

#### ➡️Attributes (Metadata)
Attributes are descriptive metadata — they provide context about the dataset or variables but don’t affect the data structure.

In [ ]:
#Reveal the attributes of the dataset
ds.attrs

These tell you:
* What the data represents (a hindcast from WAVEWATCH III).
* Where it came from (NCEP, CFSRR project).
* How it was produced (part2nc = partition-to-NetCDF conversion).
Each variable may also have its own attributes (e.g., units, standard names).
>Attributes are like the notes on the spreadsheet explaining what the numbers mean.

---
## Selecting data from the dataset
Now we'll look into how to extract, and in some cases summarize data in our dataset. First, we'll look at how to isolate and examine a single variable in our dataset. Then we'll see how we can extract values by index *position* along the different dimensions, and finally how to extract values by index *values* along the different dimensions.

### 🔍Isolating data for a specific specific *variable*
We can subset our Xarray dataset for just values related to a specific variable by calling that variable as so:

In [ ]:
#List the variables in the dataset
ds.data_vars

In [ ]:
#Select only depth records into a data array object
arr_depth = ds['depth']
type(arr_depth)

As you see, this creates an XArray `data array` object. Calling that object reveals information about its structure.

In [ ]:
#Display info on the depth data array
arr_depth

The data array object is similar to the dataset, but only retains values related to the variable. **However**, not all variables include all dimensions. For example, the `depth` variable does not have a `date` or `partition` dimension, which makes sense: it does not vary [much] across time or partition, just location. 

In [ ]:
#Show dimensions for the depth array
arr_depth.dims

The `wave_direction`, however, does include data in all the four dimensions...

In [ ]:
#Show dimensions for the wave direction array
ds['wave_direction'].dims

### 🔍Selecting data from a dataset with `.isel()`
Now, we'll focus on working with data in a data array. 

We'll begin with the `.isel()` function. This functions allows us to select values from our dataset via their *integer position* along each dimension. For, example, the value in the `wave_direction` variable at the 5th positition along the `date` axis, the 63rd position along the `latitude` axis, the 68th position along the `longitude` axis and in the 1st `partition` is **272.41 degrees**. (Note, the `values` statement returns the value(s) held in the array.)

>How do we know the value is in degrees? Have a look at the data array object: it reports the units...

In [ ]:
#Show the value at the 5th date, 63rd latitude, 68th longitude, 1st partition
ds['wave_direction'].isel(
    date=4,
    latitude=62,
    longitude=67,
    partition=0
).values

If we omit any of the dimensions (or just comment them out), the `isel()` function will return all values in the ommitted dimension.  

Here we see that, in the `wave_direction` variable, only two of the 12 partitions have values (at the specified time and spatial coordinate).

In [ ]:
#Show the value at the 5th date, 63rd latitude, 68th longitude, all partition
ds['wave_direction'].isel(
    date = 4,
    latitude=62,
    longitude=67, 
    #partition=0
).values

If we omit the spatial coordinates, we get data along all the lat/long pairs for a set time and partition, i.e., a spatial dataset which we can plot.

In [ ]:
#Save the value at the 5th date, 1st partition, all latitudes and longitudes to a variable
spatial_slice = ds['wave_direction'].isel(
    date = 4,
    #latitude=62,
    #longitude=67, 
    partition=0
).values

In [ ]:
#Plot the 2D array
plt.imshow(spatial_slice, cmap='viridis')

And yes, if we omit the time dimension, we can plot a time series for a specific location and partition.

In [ ]:
#Save a time series at 63rd latitude, 68th longitude, 1st partition to a variable
time_series = ds['wave_direction'].isel(
    #date = 4,
    latitude=62,
    longitude=67, 
    partition=0
).values

In [ ]:
#Create a simple line plot of the time series
plt.plot(time_series);

We can also request multiple values along a dimension by providing a list of indices:

In [ ]:
#Show the value at the 5th & 8th date, 63rd latitude, 68th longitude, 1st & 2nd partition
ds['wave_direction'].isel(
    date = [4,7],
    latitude=63,
    longitude=68,
    partition = [0,1]
).values

And we can extract a slice of values using the range function:

In [ ]:
#Show the value at the 5th & 8th date, 63rd latitude, 68th longitude, 1st & 2nd partition
ds['wave_direction'].isel(
    date = range(4,8),
    latitude=63,
    longitude=68,
    partition = [0,1]
).values

### Selecting values by *index* with `.sel()`
Applying the `.sel()` command to a dataset allows us to select records by their *index values*.  

Of course, to do this, we need an idea of what those index values are. Some values are intuitive, like values in the `date` dimension, but others may require more knowledge of the datasets structure.  

In [ ]:
#Show the first 10 values in the date index
ds['date'][:10].values

Looking at the date index values, we see they are datetime objects of hourly increments from Jan 1, 1979 to Jan 30, 1979.


In [ ]:
#Show the 
ds['gridpoint'].values

In [ ]:
ds['wave_direction']['partition']

In [ ]:

lon = ds.longitude.isel(date=0)
lat = ds.latitude.isel(date=0)
depth = ds.depth.isel(date=0)

coord_table = pd.DataFrame({
    'gridpoint': ds.gridpoint.values,
    'longitude': lon.values,
    'latitude': lat.values,
    'depth': depth.values
})

coord_table.head()

In [ ]:
ds = xr.open_dataset(Path.cwd().parent /'data'/'raw'/'nwio_10m.nc')
ds